# FHI-aims Interface

## Overview

The FHI-aims Interface module facilitates the conversion of **[FHI-aims](https://www.fhi-aims.org/)** (Fritz-Haber-Institute *ab initio* materials simulations) calculation outputs into the DeepH DFT data format. FHI-aims is widely recognized as one of the most accurate, all-electron, full-potential electronic structure codes, employing numerically tabulated, localized atom-centered orbitals as its basis set. This rigorous approach delivers exceptional precision for ground-state energies, electronic properties, and forces, making it a gold standard for high-accuracy materials simulations.

While the raw data output from FHI-aims can be roughly an order of magnitude larger than equivalent calculations from codes like OpenMX, this inherent complexity presents a uniquely valuable opportunity for DeepH method. The high-dimensional, information-rich nature of FHI-aims data is *ideally suited* for DeepH's machine learning framework. Training on such precise and detailed inputs allows DeepH models to capture subtle electronic interactions with greater fidelity, ultimately leading to superior predictive accuracy for large-scale systems. Consequently, the computational "overhead" of the initial FHI-aims data generation is offset by dramatically greater gains during the inference phase, where DeepH enables rapid and accurate electronic structure predictions at scales far beyond direct *ab initio* feasibility.

This converter is the product of a sustained, collaborative partnership between the DeepH team and the FHI-aims development group, ensuring robustness and long-term compatibility. While a comprehensive, [ASI-based](https://github.com/PavelStishenko/asi) interface for full periodic systems is under active development (In **newer version** of FHI-aims, the option `output_rs_matrices` can be set to output the Hamiltonian and overlap matrix files, which allows the use of the commands described in the [following section](#periodic-systems-support) for exporting **periodic systems**.), the current module provides native support for converting the Hamiltonian and overlap matrices from FHI-aims **single-atom cluster calculations**. This foundational interface allows researchers to immediately leverage their existing, high-accuracy FHI-aims datasets to train DeepH models, paving the way for next-generation machine-learning-accelerated materials discovery with unprecedented precision.

## File Structure Organization

Assuming you have already read the FHI-aims [documentation](https://fhi-aims-club.gitlab.io/tutorials/tutorials-overview/) and are familiar with conducting DFT calculations using FHI-aims, you must organize your FHI-aims data for different material structures according to the following convention.

### FHI-aims Data Structure
```bash
single_atoms_aims/
├── element_1/
│   ├── control.in
│   ├── geometry.in
│   ├── basis-indices.out
│   ├── hamiltonian.out
│   └── overlap-matrix.out
├── element_2/
│   └──...
└── ...
```

### Converted DeepH Data Structure
```bash
deeph_datasets/
├── structure_1/
│   ├── info.json
│   ├── hamiltonian.h5
│   └── overlap.h5
├── structure_2/
│   └── ...
└── ...
```

The `POSCAR` file (atomic structure) is omitted, as it is not required for single-atom systems.

## Command Line Interface

### Basic Conversion Command
You can convert FHI-aims format data to DeepH format using the command line interface:

```bash
dock convert fhi-aims single-atom-to-deeph ./fhiaims_single_atoms /tmp/deeph_single_atoms -t 0
```

Expected output:
```bash
Data: 2it [00:00, 175.85it/s]
[done] Translation completed successfully!
```

### Complete Command Line Options
For detailed parameter information, use the help command:

```bash
dock convert fhi-aims single-atom-to-deeph -h
```

```bash
Usage: dock convert fhi-aims single-atom-to-deeph [OPTIONS] AIMS_DIR DEEPH_DIR

  Translate the FHI-aims output data of single atom calculation to DeepH DFT
  data training set format.

Options:
  -t, --tier-num INTEGER  The tier number of the aims source data, -1 for
                          [aims_dir], 0 for <aims_dir>/<aims_dir>, 1 for
                          <aims_dir>/<tier1>/<data_dirs>, etc.  [default: 0]
  -h, --help              Show this message and exit.
```

## Advanced Usage
For programmatic control, DeepH-dock also provides class API:

In [ ]:
from deepx_dock.convert.fhi_aims.single_atom_aims_to_deeph import SingleAtomDataTranslatorToDeepH

aims_dir = "./fhiaims_single_atoms"
deeph_dir = "/tmp/deeph_single_atoms"
tier_num = 0

translator = SingleAtomDataTranslatorToDeepH(
    aims_dir, deeph_dir, tier_num
)
translator.transfer_all_aims_to_deeph()

Data: 2it [00:00, 175.08it/s]


## Periodic Systems Support

### Overview

In addition to single-atom calculations, this interface supports the conversion of **periodic** FHI-aims calculation outputs. This functionality is crucial for extending DeepH's capabilities to bulk materials and solid-state systems.

### Preparing FHI-aims Calculations

#### Required FHI-aims Settings
Refer to the FHI-aims [official manual](https://www.fhi-aims.org/uploads/documents/FHI-aims.250822.pdf), to obtain the real-space matrix data required by DeepH (Hamiltonian and overlap matrices), you must configure `control.in` to output these matrices. Add the following keyword:

```bash
output_rs_matrices  [format]
```

Supported formats include `plain` (text) and `hdf5` (binary).
*   **plain**: Outputs `rs_indices.out`, `rs_hamiltonian.out`(spinless), `rs_hamiltonian_up/down.out`(spin-polarized), `rs_overlap.out`, etc. (Default).
*   **hdf5**: Outputs text file `rs_indices.out`, and binary files `rs_hamiltonian.h5`(spinless), `rs_hamiltonian_up/down.h5`(spin-polarized), `rs_overlap.h5` (requires enabling HDF5 during FHI-aims compilation).

#### Additional Recommendations
**log file**: Because FHI-aims outputs its log to standard output by default, there may be no consistently named log file available from which to read the **Fermi energy**. Here we follow the [official FHI-aims tutorial](https://fhi-aims-club.gitlab.io/tutorials/basics-of-running-fhi-aims/1-Molecules/) and recommend the log file explicitly when running the calculation command, for example as shown below: 
```bash
mpirun -n 4 aims.x > aims.out 2>&1
```
In this way, the log output will be redirected to a uniformly named file **`aims.out`**, which facilitates parsing and extracting the Fermi energy.

#### Version Requirements
**Warning**: This feature relies on the `output_rs_matrices` parameter set in `control.in`, which is **only available in FHI-aims version 250822 (released ~August 2025) or later**. Older versions do not support this output. Please ensure your FHI-aims executable is updated.

#### Limitations
1.  **Spin Support**: Only non-spin-polarized and collinear spin-polarized calculations are supported. The interface **does not support** SOC calculations.
2. **Hybrid Functionals**: Currently, the interface does not support hybrid functional calculations. See [manual](https://www.fhi-aims.org/uploads/documents/FHI-aims.250822.pdf).
3. **Density Matrix and Other Advanced Features**: The interface does not currently support exporting density matrices or other advanced features that may be available in FHI-aims.

### File Structure Organization

For periodic systems (e.g., crystals, slabs), organize your data as follows:

#### FHI-aims Data Structure
```bash
periodic_aims_dataset/
├── structure_1/
│   ├── control.in             # Must contain 'output_rs_matrices'
│   ├── geometry.in
│   ├── basis-indices.out      # Generated output (text)
│   ├── rs_indices.out         # Generated output (text)
│   ├── rs_hamiltonian.out     # Generated output (or rs_hamiltonian_up/dn.h5 if spin-polarized and hdf5 format)
│   └── rs_overlap.out         # Generated output (or rs_overlap.h5 if hdf5 format)
├── structure_2/
│   └── ...
└── ...
```

#### Converted DeepH Data Structure
```bash
deeph_datasets/
├── structure_1/
│   ├── info.json
│   ├── POSCAR                 # Generated from geometry.in
│   ├── hamiltonian.h5
│   ├── overlap.h5
│   └── ...
├── structure_2/
│   └── ...
└── ...
```

### Command Line Interface

#### Basic Conversion Command
```bash
dock convert fhi-aims periodic-to-deeph ./periodic_aims_dataset /tmp/deeph_periodic_dataset -p 4
```

#### Complete Command Line Options

```bash
dock convert fhi-aims periodic-to-deeph -h
```

```bash
Usage: dock convert fhi-aims periodic-to-deeph [OPTIONS] AIMS_DIR DEEPH_DIR

  Translate the FHI-aims output data of periodic structure calculation to DeepH DFT data training set format.

Options:
  -p, --parallel-num INTEGER  The parallel processing number, -1 for using all
                              of the cores. [default: -1]
  -t, --tier-num INTEGER      The tier number of the aims source data, -1 for
                              [aims_dir], 0 for <aims_dir>/<data_dirs>, etc. [default: 0]
  --force                     Force to overwrite the existing files.
  -h, --help                  Show this message and exit.
```

### Advanced Usage (Periodic)

For programmatic access:

In [ ]:
from deepx_dock.convert.fhi_aims.aims_to_deeph import PeriodicAimsDataTranslator

# Example for programmatic usage
translator = PeriodicAimsDataTranslator(
    "./periodic_aims_dataset", 
    "/tmp/deeph_periodic_dataset",
    export_rho=False,
    export_r=False,
    n_jobs=1,
    n_tier=0,
)
translator.transfer_all_aims_to_deeph()

Data: 2it [00:00, 747.85it/s]
